In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import selenium.webdriver
from selenium import webdriver 
from datetime import date 
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import time
import warnings
warnings.filterwarnings('ignore')

## Amazon review:

In [116]:
#connecting to web driver
driver = webdriver.Chrome(r"C:\driver\chromedriver.exe")
#opening chrome
driver.maximize_window()

url = "https://www.amazon.in/"
driver.get(url)
time.sleep(3)
srch_items = ['laptops','phones','headphones','smart watches', 'Professional Cameras', 'Printers', 
              'Monitors', 'Home theater', 'Router']


title = []
review_text = []
ratings = []

for i in srch_items:
    srchbar = driver.find_element_by_id("twotabsearchtextbox")
    srchbar.send_keys(i)
    driver.find_element_by_id("nav-search-submit-button").click()  #clicking on search button
    time.sleep(1)
    item_url = []
    for i in driver.find_elements_by_xpath("//h2[@class='a-size-mini a-spacing-none a-color-base s-line-clamp-2']/a"):
        item_url.append(i.get_attribute('href'))
    for i in item_url:
        driver.get(i)
        time.sleep(1)
        for _ in range(2):
            driver.execute_script("window.scrollBy(0,6000)")
            time.sleep(1)
            
        # Clicking on see all reviews
        try:
            driver.find_element_by_xpath("//a[@class='a-link-emphasis a-text-bold']").click()
        except:
            pass
        
        url_ = []
        try:
            two_str = driver.find_element_by_xpath("//table[@id='histogramTable']/tbody/tr[4]/td[2]/a")
            url_.append(two_str.get_attribute('href'))
        except: pass
        try:
            three_str = driver.find_element_by_xpath("//table[@id='histogramTable']/tbody/tr[3]/td[2]/a")
            url_.append(three_str.get_attribute('href'))
        except: pass
        try:
            one_str = driver.find_element_by_xpath("//table[@id='histogramTable']/tbody/tr[5]/td[2]/a")
            url_.append(one_str.get_attribute('href'))
        except: pass
        try:
            fiv_str = driver.find_element_by_xpath("//table[@id='histogramTable']/tbody/tr[1]/td[2]/a")
            url_.append(fiv_str.get_attribute('href'))
        except: pass
        try:
            four_str = driver.find_element_by_xpath("//table[@id='histogramTable']/tbody/tr[2]/td[2]/a")
            url_.append(four_str.get_attribute('href'))
        except: pass
        
        for i in url_:
            driver.get(i)
            time.sleep(1)
            for i in range(10):
                ids = driver.find_elements_by_xpath("//div[@class='a-section a-spacing-none review-views celwidget']/div")
                comment_ids = []
                for i in ids:
                    comment_ids.append(i.get_attribute('id'))

                for x in comment_ids:
                    # Extract user ids from each user on a page
                    try:
                        review_title = driver.find_element_by_xpath('//*[@id="' + x +'"]/div/div/div[2]/a[2]/span[1]')
                        title.append(review_title.get_attribute("innerHTML"))
                    except NoSuchElementException:
                        title.append('')

                    try:
                        rating = driver.find_element_by_xpath('//*[@id="viewpoint-R1BI04AV972TX2"]/div[1]/div[4]/i')
                        ratings.append(rating.get_attribute("innerHTML"))
                    except NoSuchElementException:
                        ratings.append('')

                    try:
                        text = driver.find_element_by_xpath('//*[@id="' + x +'"]/div/div/div[4]/span/span')
                        review_text.append(text.get_attribute("innerHTML"))
                    except NoSuchElementException:
                        review_text.append('')
                try:
                    driver.find_element_by_xpath("//ul[@class='a-pagination']/li[2]/a").click()
                    time.sleep(3)
                except : break 
                    
                    
len(ratings), len(title), len(review_text)

(81091, 81091, 81091)

In [125]:
# Creating a dataframe
data = list(zip(review_text,ratings))
df = pd.DataFrame(data, columns = ["Review","Ratings"])
df.to_csv("amazon_rating 2.csv", index=False, encoding='utf-8')
data=pd.read_csv("amazon_rating 2.csv")

In [10]:
df = pd.read_csv("amazon_rating 2.csv")
df

,Unnamed: 0,Review,Ratings
0,0,Think twice before going for it,2.0 out of 5 stars
1,1,Overpriced,2.0 out of 5 stars
2,2,Build quality is not as good as shown in the Ad,3.0 out of 5 stars
3,3,Speed and Visuals makes this a power machine!,3.0 out of 5 stars
4,4,Overpriced and unreliable,3.0 out of 5 stars
...,...,...,...
83805,83805,Vfm. Range of Archer c6 or A6 without mu-mimo,4.0 out of 5 stars
83806,83806,USB port is not supported for printer,4.0 out of 5 stars
83807,83807,A best buy for that price range,4.0 out of 5 stars
83808,83808,Value For Money,4.0 out of 5 stars


##### Data Cleaning

In [16]:
df.isnull().sum()

Unnamed: 0        0
Review        15612
Ratings       15611
dtype: int64

In [17]:
df.dropna(inplace = True)

In [18]:
df.isnull().sum().sum()

0

In [19]:
df['Ratings'] = df['Ratings'].replace('1.0 out of 5 stars',1)
df['Ratings'] = df['Ratings'].replace('2.0 out of 5 stars',2)
df['Ratings'] = df['Ratings'].replace('3.0 out of 5 stars',3)
df['Ratings'] = df['Ratings'].replace('4.0 out of 5 stars',4)
df['Ratings'] = df['Ratings'].replace('5.0 out of 5 stars',5)

pd.to_numeric(df.Ratings)

0        2.0
1        2.0
2        3.0
3        3.0
4        3.0
        ... 
83804    4.0
83805    4.0
83806    4.0
83807    4.0
83808    4.0
Name: Ratings, Length: 68198, dtype: float64

In [20]:
df["Ratings"] = df["Ratings"].apply(pd.to_numeric).astype(int)

In [21]:
df

,Unnamed: 0,Review,Ratings
0,0,Think twice before going for it,2
1,1,Overpriced,2
2,2,Build quality is not as good as shown in the Ad,3
3,3,Speed and Visuals makes this a power machine!,3
4,4,Overpriced and unreliable,3
...,...,...,...
83804,83804,Only one problem,4
83805,83805,Vfm. Range of Archer c6 or A6 without mu-mimo,4
83806,83806,USB port is not supported for printer,4
83807,83807,A best buy for that price range,4


In [22]:
df.to_csv("Amazon_Ratings.csv")